# Solutions: Task 4.5.1 - Advanced Gradio

This notebook contains solutions to the exercises in the main notebook.

---

## Exercise Solution: Three-Column Layout

Create a layout with:
1. A narrow left sidebar (scale=1)
2. A wide center area (scale=3)
3. A narrow right sidebar (scale=1)

In [ ]:
import gradio as gr

with gr.Blocks() as three_col_solution:
    gr.Markdown("# Three Column Layout")
    
    with gr.Row():
        # Left sidebar (navigation)
        with gr.Column(scale=1):
            gr.Markdown("### Navigation")
            nav_btn1 = gr.Button("Dashboard")
            nav_btn2 = gr.Button("Analytics")
            nav_btn3 = gr.Button("Settings")
            nav_btn4 = gr.Button("Help")
        
        # Center content area (main content)
        with gr.Column(scale=3):
            gr.Markdown("### Main Content Area")
            main_input = gr.Textbox(
                label="Enter your query",
                placeholder="Type something here...",
                lines=3
            )
            submit_btn = gr.Button("Submit", variant="primary")
            main_output = gr.Textbox(label="Result", lines=10)
        
        # Right sidebar (info panel)
        with gr.Column(scale=1):
            gr.Markdown("### Info Panel")
            gr.Markdown("**Status:** Online")
            gr.Markdown("**Model:** GPT-4")
            gr.Markdown("**Tokens:** 0/4096")
            gr.Markdown("---")
            gr.Markdown("**Tips:**")
            gr.Markdown("- Be specific")
            gr.Markdown("- Ask one thing at a time")
    
    # Event handler
    def process(text):
        return f"Processed: {text}" if text else "No input provided"
    
    submit_btn.click(process, [main_input], [main_output])

three_col_solution.launch(inline=True)

In [ ]:
three_col_solution.close()

---

## Challenge Solution: Document Q&A Demo

Build a Document Q&A Demo with:
1. Three tabs: Upload, Chat, Settings
2. Custom NVIDIA-green theme
3. Session state to remember uploaded files
4. Streaming responses in the chat
5. User authentication

In [ ]:
import gradio as gr
import time

# NVIDIA theme
nvidia_theme = gr.themes.Soft(
    primary_hue="green",
    secondary_hue="gray",
    neutral_hue="slate",
).set(
    button_primary_background_fill="#76b900",
    button_primary_background_fill_hover="#5a9000",
    button_primary_text_color="white",
)

def authenticate(username, password):
    """Simple authentication."""
    valid_users = {"demo": "demo123", "admin": "admin123"}
    return valid_users.get(username) == password

with gr.Blocks(theme=nvidia_theme) as challenge_solution:
    gr.Markdown("# 📚 Document Q&A Demo")
    
    # Session state
    uploaded_files = gr.State(value=[])
    settings = gr.State(value={"model": "llama3.1:8b", "temperature": 0.7})
    
    with gr.Tabs():
        # Tab 1: Upload
        with gr.TabItem("📁 Upload"):
            gr.Markdown("### Upload Documents")
            files = gr.File(
                label="Select files",
                file_count="multiple",
                file_types=[".pdf", ".txt", ".md"]
            )
            upload_btn = gr.Button("Upload & Index", variant="primary")
            upload_status = gr.Textbox(label="Status", interactive=False)
            file_list = gr.Markdown("No files uploaded yet.")
        
        # Tab 2: Chat
        with gr.TabItem("💬 Chat"):
            chatbot = gr.Chatbot(height=400)
            msg = gr.Textbox(label="Question", placeholder="Ask about your documents...")
            with gr.Row():
                send_btn = gr.Button("Send", variant="primary")
                clear_btn = gr.Button("Clear")
        
        # Tab 3: Settings
        with gr.TabItem("⚙️ Settings"):
            gr.Markdown("### Model Settings")
            model_select = gr.Dropdown(
                choices=["llama3.1:8b", "llama3.1:70b", "mistral:7b"],
                value="llama3.1:8b",
                label="Model"
            )
            temp_slider = gr.Slider(0, 1, 0.7, step=0.1, label="Temperature")
            save_btn = gr.Button("Save Settings", variant="primary")
            settings_status = gr.Textbox(label="Status", interactive=False)
    
    # Event handlers
    def upload_files(files, current_files):
        if not files:
            return current_files, "No files selected.", "No files uploaded yet."
        
        new_files = current_files + [f.name for f in files]
        file_list_text = "**Uploaded files:**\n" + "\n".join([f"- {f}" for f in new_files])
        return new_files, f"✅ Uploaded {len(files)} file(s)!", file_list_text
    
    def streaming_response(message, history, files, settings):
        """Generate a streaming response."""
        if not files:
            history.append((message, "Please upload some documents first!"))
            yield history, ""
            return
        
        # Simulate streaming
        response = f"Based on {len(files)} documents, here's my answer about '{message}':"
        partial = ""
        
        for char in response:
            partial += char
            time.sleep(0.02)
            yield history + [[message, partial]], ""
        
        history.append((message, response))
        yield history, ""
    
    def save_settings(model, temp):
        return {"model": model, "temperature": temp}, f"✅ Settings saved: {model}, temp={temp}"
    
    upload_btn.click(
        upload_files,
        [files, uploaded_files],
        [uploaded_files, upload_status, file_list]
    )
    
    send_btn.click(
        streaming_response,
        [msg, chatbot, uploaded_files, settings],
        [chatbot, msg]
    )
    
    clear_btn.click(lambda: ([], ""), None, [chatbot, msg])
    
    save_btn.click(
        save_settings,
        [model_select, temp_slider],
        [settings, settings_status]
    )

# To launch with authentication:
# challenge_solution.launch(auth=authenticate, auth_message="Login Required")

print("Challenge solution created!")
print("To run with auth: challenge_solution.launch(auth=authenticate)")
challenge_solution.launch(inline=True)

In [ ]:
challenge_solution.close()

---

## Key Takeaways

1. **Layout flexibility**: Use `gr.Row()` and `gr.Column(scale=X)` for precise control
2. **State management**: `gr.State()` persists data across interactions
3. **Streaming**: Return a generator function for streaming outputs
4. **Authentication**: Pass an auth function to `demo.launch(auth=fn)`
5. **Themes**: Create custom themes with `gr.themes.Soft().set(...)`